## Import Libraries

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

from tqdm import tqdm

## Read Data 

In [2]:
data = pd.read_excel("../Data/dysthymia_llm_filtered_en.xlsx")

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055 entries, 0 to 1054
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  1055 non-null   object
 1   Text    1054 non-null   object
dtypes: object(2)
memory usage: 16.6+ KB


In [5]:
data = data[data['Text'].apply(lambda x: isinstance(x, str))]

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1054 entries, 0 to 1054
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  1054 non-null   object
 1   Text    1054 non-null   object
dtypes: object(2)
memory usage: 24.7+ KB


In [7]:
data.head()

,Result,Text
0,yes,"I am productive and motivated... ...fuck everything. I want to rant, I want to complain, but why even bother. Nothing’s gonna change anyway. What gives me the right to complain, I’m a shit eater too."
1,yes,when I attempt to drive it almost doesn’t feel real and I’ve gotten a little lightheaded before. my mind kind of drifts off when driving
2,no,"Hey everyone,..."
3,yes,"whenever i get the house to myself... it’s like i intentionally turn into a disgusting slob... i felt the need to hide from the world... i can never bring myself to cook, i’ll eat disgustingly... i’ve worn the same sweatpants for the past 4 days... being in this lazy isolated state feels most natural and it’s what i want all the time. i don’t care about anything."
4,yes,"I've been suicidal for the past 10 years and still can't find a good reason to stay alive. I don't feel anything – except physical pain, exhaustion, sadness and numbness. Nothing excites me. Nothing makes me 'happy' or whatever. I'm not looking forward to anything because I'm so so exhausted all the time, I don't have the energy to do anything and if I push myself I end up in a big flare up from my fibromyalgia. And it just adds such a big feeling of powerlessness."


from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Çeviri fonksiyonunu tanımla
def translate_text(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# DataFrame'deki her satır için çeviri işlemi uygula ve yeni bir kolona ekle
data['Translated_Text'] = data['Text'].apply(translate_text)

# Sonuçları göster
print(data)


In [8]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cpu"
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(57060, 1024, padding_idx=57059)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(57060, 1024, padding_idx=57059)
      (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1

In [9]:
def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating Process"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts, batch_size=16)

data['Translated_Text'] = translated_texts

print(data)

Translating Process: 100%|███████████████████████████████████████████████████████████| 66/66 [1:06:57<00:00, 60.87s/it]

     Result  \
0       yes   
1       yes   
2        no   
3       yes   
4       yes   
...     ...   
1050    yes   
1051    yes   
1052    yes   
1053    yes   
1054    yes   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Text  \
0                                                                                                                                                                                                                                                                                   I am productive and motivated... ...fuck everything. I want to r

In [10]:
data.to_excel("../Data/dysthymia_to_tr.xlsx", index=False)

In [ ]:
data.head()

data.to_csv("agoraphobia_translated_to_tr", index=False)

df = pd.read_csv("agoraphobia_translated_to_tr")